In [2]:
import re
from itertools import groupby
from datetime import datetime, timedelta
from asgiref.sync import sync_to_async
import django
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'oogway.settings')
django.setup()
from PostAnalyzer.models import (
    Market,
    Symbol,
    PositionSide,
    MarginMode,
    EntryTarget
)

In [3]:
def returnValue(val):
    return val.group(1) if val else None

In [4]:
def isPredictMsg(msg):
    patterns = [
        r"Symbol:\s*#?([A-Z0-9]+)[/\s]?USDT",
        r"Take-Profit Targets:([\s\S]+?)(StopLoss|Description)",
        r"Entry Targets:([\s\S]+?)Take-Profit",
        r"Market:\s*([A-Z]+)",
        r"(StopLoss|Description):\s*([\d.]|\w+)",
    ]

    # Check if all patterns have a value
    return all(re.search(pattern, msg, re.IGNORECASE) for pattern in patterns)

In [5]:
async def findPosition(msg):
    position_match = re.search(r"Position:\s*([A-Z]+)", msg)
    
    try:
        position_value = await sync_to_async(PositionSide.objects.get)(name=returnValue(position_match).upper())
        return position_value
    except:
        return None

In [6]:
async def findMarket(msg):
    market_match = re.search(r"Market:\s*([A-Z]+)", msg, re.IGNORECASE)
    
    try:
        market_value = await sync_to_async(Market.objects.get)(name=returnValue(market_match).upper())
        return market_value
    except:
        return None


In [7]:
async def findLeverage(msg):
    leverage_match = re.search(r"Leverage:\s*(Isolated|Cross)\s*(\d+x)", msg, re.IGNORECASE)
    if leverage_match:
        leverage_type = await sync_to_async(MarginMode.objects.get)(name=returnValue(leverage_match).upper())   
        leverage_value = int(leverage_match.group(2).lower().replace("x",""))    
    else:
        leverage_type = None
        leverage_value = None
    
    return leverage_type, leverage_value

In [8]:
def findStopLoss(msg):
   return returnValue(re.search(r"StopLoss:\s*([\d.]+)", msg))

In [9]:
def findEntryTargets(msg):
    match = re.search(r"Entry Targets:([\s\S]+?)Take-Profit", msg, re.IGNORECASE)
    if match:
        extracted_data = returnValue(match)
        return [float(x.strip()) for i, x in enumerate(re.findall(r"(\d+\.\d+|\d+)", extracted_data))  if i % 2 == 1]
        

In [10]:
def findTakeProfits(msg):
    match = re.search(r"Take-Profit Targets:([\s\S]+?)(StopLoss|Description)", msg, re.IGNORECASE)
    if match:
        extracted_data = returnValue(match)
        return [float(x.strip()) for i, x in enumerate(re.findall(r"(\d+\.\d+|\d+)", extracted_data)) if i % 2 == 1]
        

In [16]:
async def findSymbol(msg):
    symbol_match = re.search(r"Symbol:\s*#?([A-Z0-9]+)[/\s]?USDT", msg, re.IGNORECASE)

    try:
        symbol_value = await sync_to_async(Symbol.objects.get)(asset=returnValue(symbol_match).upper())
        return symbol_value
    except:
        return None

In [19]:
msg = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #AVAX/USDT\nMarket: FUTURES\nPosition: LONG\nLeverage: Isolated 3x\n\nEntry Targets: \n1) 11.80\n2) 11.502\n\nTake-Profit Targets: \n1) 12.025\n2) 12.281\n3) 12.564\n4) 12.950\n\nStopLoss: 11.19\nB.Z",    "edit_date": None,
    "media": None,
}

msg1 = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #FILUSDT.P\nMarket: FUTURES\nPosition: SHORT\nLeverage: Isolated 4x\n\nEntry Targets: \n1) 5.430\n2) 5.212\n\nTake-Profit Targets: \n1) 5.517\n2) 5.608\n3) 5.715\n4) 5.823\n5) 5.982\n\nStopLoss: 5.154\nB.Z",    
    "edit_date": None,
    "media": None,
}

msg3 = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #BTC/USDT Market: FUTURES Position: LONG Leverage: Isolated 6x Entry Targets: 1) 30000.54 2) 32000.43 3) 33000. Take-Profit Targets: 1) 30010.66 2) 30016 3) 30019 4) 30040 StopLoss: 29010 B.Z #test",    
    "edit_date": None,
    "media": None,
}

msg4 = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #Galausdt Market: FUTURES Position: LONG Leverage: Isolated 6x Entry Targets: 1) 30000.54 2) 32000.43 3) 33000. Take-Profit Targets: 1) 30010.66 2) 30016 3) 30019 4) 30040 StopLoss: 29010 B.Z #test",    
    "edit_date": None,
    "media": None,
}

msg5 = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #1000BONKUSDT.P\nMarket: FUTURES\nPosition: LONG\nLeverage: Isolated 4x\n\nEntry Targets: \n1) 0.022713\n2) 0.021730\n\nTake-Profit Targets: \n1) 0.23111\n2) 0.23689\n3) 0.24366\n4) 0.25361\n\nStopLoss: 0.21400\nB.Z",    
    "edit_date": None,
    "media": None,
}

msg6 = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #GTCUSDT.P\nMarket: FUTURES\nPosition: LONG\nLeverage: Isolated 4x\nOrder Type: Limit\n\nEntry Price: \n1) 1.139\n2) 1.096\n\nTake-Profit Targets: \n1) 1.155\n2) 1.178\n3) 1.205\n4) 1.234\n5) 1.271\n\nStopLoss: 1.079\nB.Z",    
    "edit_date": None,
    "media": None,
}

# spot
msg2 = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #ETH/USDT\nMarket: SPOT\n\nEntry Targets: \n1) 2930.00\n2) 2870.00\n\nTake-Profit Targets: \n1) 3100.00\n2) 3200.00\n\n\nDescription: HOLD MODE\nمیان مدت\nB.Z",    
    "edit_date": None,
    "media": None,
}

string = msg6["message"]


# ************* FIND IS PREDICT *************
isPredict = isPredictMsg(string)
print(isPredict)


# ************* FIND SYMBOL *************
# method 1
# symbol_match = re.search(r"Symbol:(.+)", string)
# symbol_match = returnValue(symbol_match).split("USDT")[0].replace("/", "").replace(" ", "").replace("#", "")
 
# method 2
symbol = await findSymbol(string)
print(symbol)


# ************* FIND MARKET *************
market = await findMarket(string)
print(market.name)

if market.name != "SPOT":
    # ************* FIND POSITION *************
    position = await findPosition(string)
    print(position.name)

    # ************* FIND LEVERAGE *************
    marginMode, leverage = await findLeverage(string)
    print(marginMode.name, leverage)


# ************* FIND STOP LOSS *************
stopLoss = findStopLoss(string)
print(stopLoss)

# ************* FIND ENTRY TARGETS *************
entries = findEntryTargets(string)
print(entries)
 
# ************* FIND TAKE PROFITS *************
profits = findTakeProfits(string)
print(profits)

False
GTC-USDT
FUTURES
LONG
ISOLATED 4
1.079
None
[1.155, 1.178, 1.205, 1.234, 1.271]


In [13]:
# from bingx.api import BingxAPI
# from dotenv import dotenv_values
# from bingX.spot import Spot
# from bingX.perpetual.v2 import Perpetual

# # ****************************************************************************************************************************

# config = dotenv_values(".env")


# API_KEY = config["API_KEY"]
# SECRET_KEY = config["SECRET_KEY"]

# client = Spot(API_KEY, SECRET_KEY)
# client1 = Perpetual(API_KEY, SECRET_KEY)

# client.symbols()
# client1.trade_order()

ModuleNotFoundError: No module named 'bingX'

In [ ]:
from bingx.api import BingxAPI
from dotenv import dotenv_values

config = dotenv_values(".env")


API_KEY = config["API_KEY"]
SECRET_KEY = config["SECRET_KEY"]

bingx =  BingxAPI(API_KEY, SECRET_KEY, timestamp="local")


entry_price = 2.6902  
stop_loss = 2.5460     
take_profit = 2.73  
symbol = "PHB-USDT" 
time_interval = "1m"
start_timestamp = 1717063704000

# 86400000 == milliseconds in a day
next_day = 1717063704000 + 86400000
1717063800000
# res = bingx.get_kline_data(symbol,time_interval ,1718442480000 ,1718528880000,1440)

reached_target = None
stop_loss_reached = None
entry_reached = None
take_profit_reached = None
finish = True
isEntrySuccess = False
i = 1

while finish:
    res = bingx.get_kline_data(symbol,time_interval ,start_timestamp ,next_day, 1440)
    for row in reversed(res):

        if not entry_reached and float(row['low']) <= entry_price:
            entry_reached = row
       
        if bool(entry_reached):
            if not bool(stop_loss_reached) and float(row['low']) <= stop_loss:
                if not reached_target:
                    reached_target = 'Stop-Loss'
                stop_loss_reached = row
            
            if not bool(take_profit_reached) and float(row['high']) >= take_profit:
                if not reached_target:
                    reached_target = 'Take-Profit'
                take_profit_reached = row
          

        if bool(take_profit_reached and stop_loss_reached):
            finish = False
            break
    start_timestamp = next_day
    next_day = start_timestamp + 86400000
    print(i)
    i += 1


if reached_target:
    print(f"The {reached_target} was reached first.")
else:
    print("Neither stop-loss nor take-profit was reached within the available data.")

print(take_profit_reached)
print(stop_loss_reached)
print(entry_reached)

# The Take-Profit was reached first.
# Take-Profit  {'open': '2.7291', 'close': '2.7361', 'high': '2.7361', 'low': '2.7244', 'volume': '19567.00', 'time': 1717155240000}
# Stop-Loss    {'open': '2.5846', 'close': '2.5376', 'high': '2.5949', 'low': '2.5333', 'volume': '14267.00', 'time': 1717783200000}
# Entry        {'open': '2.7033', 'close': '2.6899', 'high': '2.7033', 'low': '2.6848', 'volume': '53505.00', 'time': 1717132440000}




1
2
3
4
5
6
7
8
9
The Take-Profit was reached first.
{'open': '2.7291', 'close': '2.7361', 'high': '2.7361', 'low': '2.7244', 'volume': '19567.00', 'time': 1717155240000}
{'open': '2.5846', 'close': '2.5376', 'high': '2.5949', 'low': '2.5333', 'volume': '14267.00', 'time': 1717783200000}
{'open': '2.7033', 'close': '2.6899', 'high': '2.7033', 'low': '2.6848', 'volume': '53505.00', 'time': 1717132440000}


In [55]:
import requests
import pandas as pd
from bingx.api import BingxAPI
from dotenv import dotenv_values

config = dotenv_values(".env")


API_KEY = config["API_KEY"]
SECRET_KEY = config["SECRET_KEY"]

bingx =  BingxAPI(API_KEY, SECRET_KEY, timestamp="local")

# Get historical data from Binance
def get_binance_data(symbol, interval, limit=100):
    url = f"https://api.binance.com/api/v3/klines"
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': limit
    }
    response = requests.get(url, params=params)
    data = response.json()
    print(data)
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['close'] = df['close'].astype(float)
    return df

# Calculate RSI
def calculate_rsi(df, period=14):
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Get RSI
symbol = 'BTCUSDT'
interval = '1h'
df = get_binance_data(symbol, interval)
df['RSI'] = calculate_rsi(df)
print(df[['timestamp', 'close', 'RSI']].tail())


[[1718805600000, '65090.22000000', '65141.89000000', '64832.57000000', '64952.02000000', '1222.80938000', 1718809199999, '79449880.73130450', 67177, '640.20275000', '41599059.35256780', '0'], [1718809200000, '64952.02000000', '65129.57000000', '64756.28000000', '64922.00000000', '798.50963000', 1718812799999, '51839808.77975460', 52512, '403.81843000', '26218420.01792790', '0'], [1718812800000, '64921.99000000', '65100.03000000', '64807.35000000', '64922.26000000', '801.58932000', 1718816399999, '52079417.08972980', 48739, '413.75923000', '26881032.87600560', '0'], [1718816400000, '64922.26000000', '65284.37000000', '64894.25000000', '65083.99000000', '873.54080000', 1718819999999, '56877824.06835860', 73152, '480.70386000', '31299356.63932710', '0'], [1718820000000, '65083.99000000', '65158.79000000', '64964.06000000', '64973.00000000', '464.92693000', 1718823599999, '30252689.16834880', 33953, '202.16433000', '13154275.42908380', '0'], [1718823600000, '64973.00000000', '64985.4900000

In [71]:
# RSI
import pandas as pd

def calculate_rsi(data, period=14):
    df = pd.DataFrame(data)
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    
    # Adjust the rolling mean calculation for small sample sizes
    avg_gain = gain.rolling(window=min(len(gain), period), min_periods=1).mean()
    avg_loss = loss.rolling(window=min(len(loss), period), min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Example data


res = bingx.get_kline_data("BTC-USDT","1h" ,1716478063000 ,1719156463000, 1440)
rsi_values = calculate_rsi(res)
# print(rsi_values)

for x in rsi_values:
    print(x)


nan
0.0
53.07473982970666
63.804427146679444
71.48879095612139
75.07537688442193
70.27281279397886
66.96548632900009
71.32295719844309
69.16111180983492
70.39362472832612
67.2899353647275
66.89615605278256
67.05492748658205
68.3592893178327
83.70085170925245
75.09573304157558
71.80678123548535
47.142094017094294
45.84017515051971
48.69200751553701
48.419085944236826
40.589306406224246
40.36878498518266
36.213078367765
45.18712207877093
44.57520554570348
42.82119830162011
39.33388842631166
29.35815396548385
30.530348860657597
34.79083853870942
59.90190067443251
44.90259740259729
68.04744869139475
61.462585034013856
60.357265544486225
56.7781547907579
54.854823602872656
47.90519112001347
38.99456521739131
35.23634131368938
42.11625096143273
42.514185945001834
40.15253014531546
44.82660332541557
45.8201156932264
44.159697868896615
19.271436848203905
20.478682468831636
24.74588665447908
46.28743181565174
35.053110773899775
40.681542850443535
42.32911804171926
46.817082997582595
48.98353559

In [127]:
# RSI
from bingx.api import BingxAPI
from dotenv import dotenv_values

config = dotenv_values(".env")


API_KEY = config["API_KEY"]
SECRET_KEY = config["SECRET_KEY"]

bingx =  BingxAPI(API_KEY, SECRET_KEY, timestamp="local")



import pandas as pd
import numpy as np
import codecs, json 

def rsi_tradingview(ohlc: pd.DataFrame, period: int = 14, round_rsi: bool = True):
    """ Implements the RSI indicator as defined by TradingView on March 15, 2021.
        The TradingView code is as follows:
        //@version=4
        study(title="Relative Strength Index", shorttitle="RSI", format=format.price, precision=2, resolution="")
        len = input(14, minval=1, title="Length")
        src = input(close, "Source", type = input.source)
        up = rma(max(change(src), 0), len)
        down = rma(-min(change(src), 0), len)
        rsi = down == 0 ? 100 : up == 0 ? 0 : 100 - (100 / (1 + up / down))
        plot(rsi, "RSI", color=#8E1599)
        band1 = hline(70, "Upper Band", color=#C0C0C0)
        band0 = hline(30, "Lower Band", color=#C0C0C0)
        fill(band1, band0, color=#9915FF, transp=90, title="Background")

    :param ohlc:
    :param period:
    :param round_rsi:
    :return: an array with the RSI indicator values
    """
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    delta = ohlc["close"].diff()

    up = delta.copy()
    up[up < 0] = 0
    up = pd.Series.ewm(up, alpha=1/period).mean()

    down = delta.copy()
    down[down > 0] = 0
    down *= -1
    down = pd.Series.ewm(down, alpha=1/period).mean()

    rsi = np.where(up == 0, 0, np.where(down == 0, 100, 100 - (100 / (1 + up / down))))

    return np.round(rsi, 2) if round_rsi else rsi


# [1704122863000,1706801263000,1709306863000,1711985263000,1711985263000,1717255663000]

start_timestamp = 1704122863000
current_timestamp = int(datetime.now().timestamp() * 1000)
bool = True
res = []
while bool:
    data = list(bingx.get_kline_data("BTC-USDT","1h" ,start_timestamp ,limit= 1440))
    # data = list(reversed(bingx.get_kline_data("BTC-USDT","1h" ,start_timestamp ,limit= 1440)))
    start_timestamp = data[0]['time'] +1
    res += list(reversed(data))
    print(start_timestamp)
    # start_timestamp = data[0]['time']
    # start_timestamp = data[len(data)-1]['time']
    if len(data) < 1440: 
        break
#    if(start_timestampint(datetime.now().timestamp() * 1000))
# print(res[0])
df = pd.DataFrame(reversed(res))
print(df.__len__())
df['close'] = pd.to_numeric(df['close'], errors='coerce')
array_list = rsi_tradingview(df)
b = array_list.tolist()
json.dump(b, codecs.open("test.json", 'w', encoding='utf-8'), 
          separators=(',', ':'), 
          sort_keys=True, 
          indent=4)

json.dump(res, codecs.open("test1.json", 'w', encoding='utf-8'), 
          separators=(',', ':'), 
          sort_keys=True, 
          indent=4)
print()




1709305200001
1714489200001
1719180000001
4183

